In [ ]:
### Load the dataset to Colab VM for faster operations ###

!cp drive/MyDrive/CS298/malware_data.zip .
!unzip malware_data.zip
!rm malware_data.zip
!rm -rf /content/__MACOSX

In [2]:
### Load the mnemonic opcode to integer mappings ###

!cp drive/MyDrive/CS298/data.py .
!cp drive/MyDrive/CS298/opdictWinWebSec.json .
!cp drive/MyDrive/CS298/opdictZbot.json .
!cp drive/MyDrive/CS298/opdictVBInject.json .
!cp drive/MyDrive/CS298/opdictOnLineGames.json .
!cp drive/MyDrive/CS298/opdictRenos.json .

In [9]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import json
import tensorflow
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import precision_score, recall_score

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

In [14]:
class GAN_Test():
    def __init__(self, family, num_samples, num_unique, initial, limit, increment, architecture):
        self.family = family
        self.num_samples = num_samples
        self.latent_dim = 100
        self.factor = num_unique/2.
        self.initial = initial
        self.limit = limit
        self.increment = increment
        self.architecture = architecture
        print('Family: %s, Num_samples: %d, Factor: %f, (Initial, Limit, Increment) : (%d,%d,%d), Architecture: %s' % (self.family, self.num_samples, self.factor, self.initial, self.limit, self.increment, self.architecture))
        
    def generate(self, generator):
        noise = np.random.normal(0, 1, (self.num_samples, self.latent_dim))
        gen_samples = generator.predict(noise)
        gen_samples = (gen_samples + 1) * self.factor
        #print(gen_samples.shape)
        gen_samples = np.rint(gen_samples)
        gen_samples = gen_samples.astype(int)
        gen_samples = np.reshape(gen_samples, (self.num_samples, 600))
        return gen_samples
    
    def test_runner(self, real_data):
        idx = np.random.randint(0, real_data.shape[0], self.num_samples)
        real = real_data[idx]

        while(self.initial <= self.limit):
            generator = load_model('/content/drive/MyDrive/CS298/%s_Results/Models_test/%s/%s_generator_%d.hdf5' % (self.architecture, self.family, self.family, self.initial), custom_objects={'LeakyReLU': LeakyReLU})
            #print(generator.get_weights())
            #generator.summary()
            gen_samples = self.generate(generator)
            #print(gen_samples[:2])
            #print(real[:2])
            X = np.concatenate([real, gen_samples])
            y= [1]*self.num_samples + [0]*self.num_samples

            trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, shuffle=True)

            model = GaussianNB()
            clf = RandomForestClassifier(n_estimators=25, max_depth=3)
            svmclf = SVC(C=5, kernel='rbf')
            
            model.fit(trainX, trainy)
            clf.fit(trainX, trainy)
            svmclf.fit(trainX, trainy)

            gb_acc = model.score(testX, testy)
            rf_acc = clf.score(testX, testy)
            svm_acc = svmclf.score(testX, testy)

            #rf_pred = clf.predict(textX)
            #gb_pred = model.predict(textX)
            svm_pred = svmclf.predict(testX)


            print('[Epoch Number: %d, GB Score: %f, RF Score: %f, SVM Score: %f, SVM Precision: %f, SVM Recall: %f]' % (self.initial, gb_acc, rf_acc, svm_acc, precision_score(testy, svm_pred), recall_score(testy, svm_pred)))
            #print('[Epoch Number: %d, SVM Precision: %f, SVM Recall: %f]' % (self.initial, precision_score(testy, svm_pred), recall_score(testy, svm_pred)))
            self.initial += self.increment
       

In [ ]:
files = glob.glob('Renos/*.txt')
print(len(files))

1568


In [ ]:
import numpy as np
import data
#family = ['WinWebSec', 'Zbot', 'VBInject', 'OnLineGames', 'Renos', 'BHO', 'zeroaccess']
family = ['WinWebSec']

for fam in family:
  initial = 0
  data_obj = data.DATA(fam, 600)
  data_samples, num_unique = data_obj.load_data(True)
  print('Num unique: %d'%num_unique)
  gt = GAN_Test(fam, 32, num_unique, 0, 1500, 500, 'WGANGP')
  gt.test_runner(data_samples)


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [ ]:

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
print(tagged_data)

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']), TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']), TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']), TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True